In [10]:
import cv2
import numpy
from keras.models import load_model
import os
from PIL import Image
import matplotlib.pyplot as plt

categories = ["0", "1"] # 0 -> false, 1 -> true
cnn_model = load_model('handgesture_detection.h5')

# 살색 범위
lower = numpy.array([0, 48, 80], dtype = "uint8")
upper = numpy.array([20, 255, 255], dtype = "uint8")

blue_color = (255, 0, 0)
green_color = (0, 255, 0)
red_color = (0, 0, 255)
white_color = (255, 255, 255)

cam_capture = cv2.VideoCapture(0) # 카메라 시작
str = "hand gesture detection"

# 사각형 좌표
upper_left = (50, 50)
bottom_right = (300, 300)

while True:
    _, image_frame = cam_capture.read() # 영상의 한 프레임씩 읽음, image_frame에 읽은 프레임이 나옴
    image_frame = cv2.flip(image_frame, 1) # 1 : 좌우반전 2 : 상하반전
    cv2.putText(image_frame, str, (10, 35), cv2.LINE_AA, 1, blue_color, 2) # 문자열 화면에 출력

    # Rectangle(이미지, 시작좌표, 종료좌표, 색상, 선 두께)
    r = cv2.rectangle(image_frame, upper_left, bottom_right, blue_color, 5) # 영상에서 사각형(ROI) 그리기
    rect_img = image_frame[upper_left[1]: bottom_right[1], upper_left[0]: bottom_right[0]] # 사각형의 프레임을 반환
    image_frame[upper_left[1]: bottom_right[1], upper_left[0]: bottom_right[0]] = rect_img # 반환받은 프레임을 사각형에
    # rect_img를 연속적으로 모델에 입력
    try:
        img2 = cv2.cvtColor(rect_img, cv2.COLOR_BGR2HSV) 
        mask_img2 = cv2.inRange(img2, lower, upper) # 살색 부분 검출
        f_numpy = numpy.array(mask_img2, 'uint8')
        f_numpy = f_numpy/255.0
        resize_f = cv2.resize(f_numpy, dsize=(28, 28), interpolation=cv2.INTER_AREA) # 이미지 리사이징
        cv2.imwrite(os.path.join(os.getcwd(), 'recognition_img'+ '.jpg'), resize_f) # 이미지 저장
        img_numpy = numpy.array(resize_f, 'uint8')
        test_img = img_numpy.reshape(1, 28, 28, 1) # 모델 사이즈에 맞게 리쉐이프
        prediction = cnn_model.predict_classes(test_img)
        if categories[prediction[0]]=='1': # 손 동작이 참값과 일치하면
            cv2.putText(image_frame, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, green_color, 2)
            cv2.imshow("UTH_pro", image_frame)
        else:
            cv2.putText(image_frame, "locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, red_color, 2)
            cv2.imshow("UTH_pro", image_frame)
    except:
        pass
    
    if cv2.waitKey(1) == ord('q'): 
        cam_capture.release()
        cv2.destroyAllWindows()


TypeError: 'NoneType' object is not subscriptable